In [ ]:
# Importing dependencies
import numpy as np
import cv2
import open3d as o3d
from tqdm import tqdm 
import os
import time

import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib import rcParams

import os.path as osp
from nuscenes import NuScenes

# Utils to use NuScenes
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import RadarPointCloud
from utils.misc import *

%matplotlib inline

In [ ]:
!mkdir -p /users/CeliaSagastume/Desktop/Lidar-Net/data/sets/nuscenes
!tar -xf v1.0-mini.tgz -C /users/CeliaSagastume/Desktop/Lidar-Net/data/sets/nuscenes  # Uncompress the nuScenes mini split.
!tar -xf nuScenes-lidarseg-mini-v1.0.tar.bz2 -C /users/CeliaSagastume/Desktop/Lidar-Net/data/sets/nuscenes   # Uncompress the nuScenes-lidarseg mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.


In [ ]:
%matplotlib inline
nuscenes_root = "/users/CeliaSagastume/Desktop/Lidar-Net/data/sets/nuscenes"

from nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot=nuscenes_root, verbose=True)

In [ ]:
#Function for Obtain all annotation boxes from dataset with corresponding data from sensor

def Get_Data(sensor):

    # Define sensor data to be saved 
    Sensor_data = sensor

    # Define number of samples from dataset total
    samples = np.array(nusc.sample).shape

    # Define path for point clouds 
    pc_folder = os.getcwd() + '/dataset/lidar/'

    # Define path for JSON information files 
    sample_folder = os.getcwd() + '/dataset/samples/'

    # Define path for segmentation files 
    pcd_segmentation_path =  os.getcwd() + '/dataset/pcd_segmentation/'

    for sample_idx in tqdm(range(samples[0]), desc = "Sample files"):

        # Define current sample
        current_sample = nusc.sample[sample_idx]

        # Get point cloud for current sample 
        point_data = nusc.get('sample_data', current_sample['data'][sensor_laser])

        # Get all annotations for current sample 
        annotations = current_sample['anns']

        # Create dictionary to store all inormation in JSON for each sample
        data_json = {}
        data_json['instance'] = []

        # Loop over all the annotations
        for ann in annotations:
            # Obtain current annotation
            annotation_metadata =  nusc.get('sample_annotation', ann)

            category_name = annotation_metadata['category_name'].replace(".", "_")

            # Exclude samples that do not have visibility > 80% (category 4)
            if (int(annotation_metadata['visibility_token'])) < 4 : 
                continue

            # Obtain the camera channel
            camera_channel = get_camera_data(nusc, annotation_metadata['token'])

            # Extract the sample_data giving the interest camera
            cam_data = nusc.get('sample_data', current_sample['data'][camera_channel])

            # Additional filter to detect if the frame is a keyframe
            if cam_data['is_key_frame'] != True:
                continue

            # Obtain the target for the given instance 
            target_box = target_to_cam(nusc, point_data['token'], ann, pointsensor_channel = sensor_laser)

            # Compute the 2d bounding box of the current annotation
            bbox = bbox_3d_to_2d(nusc, camera_token = cam_data['token'], annotation_token = ann, visualize = False)

            # Break the loop if the bbox is not intersected with the current image channel
            if bbox is None:
                continue

            # Map the velo pointcloud to the interest instance
            points, coloring, ori_points, im = map_pointcloud_to_image_(nusc, bbox, point_data['token'], 
                                                                        cam_data['token'], dist_thresh = 0.08,
                                                                        visualize = False)


            # If the pointcloud is empty or there are less than 20 points, continue
            if np.array(ori_points).shape[1] == 0 or np.array(ori_points).shape[1] < 100:
                continue        

            # Dataroot for PointCloud .txt file using the annotation token
            pc_path = pc_folder + 'pc_anno_' + annotation_metadata['token'] + '.txt'

            # save the Pointcloud in a .txt file
            save_to_txt(pc_path, ori_points)

            #  PointCloud in camera frame plus coloring row
            points_coloring = np.vstack([points, np.transpose(coloring)])

            # Append the information for each annotation
            data_json['instance'].append({
                'annotation_token': annotation_metadata['token'],
                'sample_token' : current_sample['token'],
                'pointcloud_path': pc_path,
                'pcl_shape': np.array(ori_points).shape,
                'pcd_path': pcd_segmentation_path+'pcd_segmentation_'+annotation_metadata['token']+'.pcd',
                'position_coord': [target_box.center[0], target_box.center[1], target_box.center[2]],
                'wlh_values': annotation_metadata['size'],
                'orientation_value': target_box.orientation.degrees,
                'rotation_axis': target_box.orientation.axis[2],
                'category': category_name
            })

        file_name= 'sample_'+ current_sample['token'] + '_file.txt'
        # Save the dictionary in a .txt file in JSON format for each sample       
        save_in_file(sample_folder + file_name, data_json)    


In [ ]:
Get_Data('LIDAR_TOP')

In [ ]:
Dataset and Instance segmentation adapted from:

@misc{salazargomez2021highlevel,
      title={High-level camera-LiDAR fusion for 3D object detection with machine learning},
      author={Gustavo A. Salazar-Gomez and Miguel A. Saavedra-Ruiz and Victor A. Romero-Cano},
      year={2021},
      eprint={2105.11060},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}